In [45]:
# for dataframe and numerical computational
import pandas as pd
import numpy as np

# selenium driver for sraping
from selenium import webdriver
# for importing time
import time

from bs4 import BeautifulSoup
 

from selenium.webdriver.common.keys import Keys

# wait untill event is ready
from selenium.webdriver.support.wait import WebDriverWait

# expected conditon
from selenium.webdriver.support import expected_conditions as EC

# by features from web driver
from selenium.webdriver.common.by import By

import logging
logging.basicConfig(filename='test.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')
class LinkedScrape:
    
    def __init__(self,loginEmail=None,Password=None,Keywords=None,location=None,filename="Database.cvs"):
        self.loginEmail = loginEmail
        self.Password = Password
        self.Keywords = Keywords
        self.location = location
        self.sleeptime = 3
        self.filename = filename
        
        
        self.driver = webdriver.Chrome(executable_path = 'chromedriver_linux64/chromedriver')
        
        logging.info("Email and Password and keywords and location are setting up!........")
        
    def loginlinkedin(self):
        
        logging.info("loginlinkedin() setting their parameters")
        
        self.driver.get('https://www.linkedin.com/login')
        
        # introduce email and password and hit enter
        login_email = self.driver.find_element(By.NAME, 'session_key')
        login_email.clear()
        login_email.send_keys(self.loginEmail)
        login_pass = self.driver.find_element(By.NAME,'session_password')
        login_pass.clear()
        login_pass.send_keys(self.Password)
        login_pass.send_keys(Keys.RETURN)
        print("Login Successfull :)")
        
    def SearchingKeywordLocation(self):
        
        logging.info("Entering keywords and location are underprocessing.......")
        # go to Jobs
        jobs_link = self.driver.find_element(By.LINK_TEXT,'Jobs')
        jobs_link.click()

        time.sleep(3)

        # search based on keywords and location and hit enter
        search_keywords = self.driver.find_element(By.CSS_SELECTOR,".jobs-search-box__text-input[aria-label='Search by title, skill, or company']")
        time.sleep(self.sleeptime)

        search_keywords.clear()
        time.sleep(self.sleeptime)
        search_keywords.send_keys(self.Keywords)
        time.sleep(self.sleeptime)
        search_location = self.driver.find_element(By.CSS_SELECTOR,".jobs-search-box__text-input[aria-label='City, state, or zip code']")
        time.sleep(self.sleeptime)
        search_location.clear()
        time.sleep(self.sleeptime)
        search_location.send_keys(self.location)
        time.sleep(self.sleeptime)
        search_location.send_keys(Keys.RETURN)
        print("Searching keywords and location is successfull :)")
        
        
    def SearchingForJobs(self):
        logging.info("SearchingForJobs() has been called")
        Database = {
        'JobTitle' : [],
        'CompanyName' : [],
        'location': [],
        'link':[],
        'space':[],
        'Jobdetail':[]}

        # we can increase the range of page for example i set here is 3
        for page in range(1,3):
            
            
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            logging.info("I am on Page Number {}.....".format(page)) 
            print("I am on Page Number {}.....".format(page)) 
            
            time.sleep(2)
            jobWindow = self.driver.page_source
            # backdriver = driver
            JobPageSoup = BeautifulSoup(jobWindow,'html.parser')


            jobparser = JobPageSoup.find_all('div' , 'flex-grow-1 artdeco-entity-lockup__content ember-view') # return list

            storelink = self.driver.current_url
            logging.info("Data Scrapping..... on page {} ".format(page))
            for  job in range(0,len(jobparser)):
                link = 'https://www.linkedin.com/'+ str(jobparser[job].div.a.get('href'))
                
                self.driver.get(link)
                time.sleep(2)

                JobDetail = self.driver.page_source
                # print(JobDetail)
                # JobDetai

                SoupJobParser = BeautifulSoup(JobDetail,'html.parser')
                try:
                    CompanyName = SoupJobParser.find('div','p5').find('span','jobs-unified-top-card__company-name').a.get_text().strip()
                except:
                    CompanyName = "Not available"

                try:
                    JobTitle = SoupJobParser.find('div','p5').h1.get_text().strip()
                    print("On Job title : {} ".format(JobTitle))
                except:
                    JobTitle = "Not available"

                try:
                    location = SoupJobParser.find('span','jobs-unified-top-card__bullet').get_text().strip()    
                except:
                    location = "Not available"
                try:
                    space = SoupJobParser.find('span','jobs-unified-top-card__workplace-type').get_text().strip()
                except:
                    space = "Not available"

                try:
                    # for searching experience requirement 
                    Jobdetail = SoupJobParser.find('article','jobs-description__container').find('div','jobs-description__content').span.get_text().strip('\n')
                except:
                    Jobdetail =  "Not available"

                Database['CompanyName'].append(CompanyName)
                Database['JobTitle'].append(JobTitle)
                Database['location'].append(location)
                Database['link'].append(link)
                Database['space'].append(space)
                Database['Jobdetail'].append(Jobdetail)

            self.driver.get(storelink)
            path = "//button[@aria-label='Page {}']".format(page)

            n_page = self.driver.find_element(By.XPATH,str(path)).click()
        
        
        df = pd.DataFrame.from_dict(Database)
        df.to_csv(self.filename)
        
        print("After all we scraped all the Data...:)")
    

    
if __name__ == "__main__":
    loginEmail = 'p180054@nu.edu.pk'
    Password = 'razerblade123!@#'
    Keywords = "Python Development"
    location = "Pakistan"
    filename = "Database.csv"
    
    
    LinkedScrapeObject = LinkedScrape(loginEmail,Password,Keywords,location)            
    
    try:
        LinkedScrapeObject.loginlinkedin()
    except:
        print("Error in loginlinkedin() method.")
        
    

/tmp/ipykernel_49290/3673022840.py:38: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(executable_path = 'chromedriver_linux64/chromedriver')


Login Successfull :)


In [49]:
LinkedScrapeObject.SearchingKeywordLocation()
     
    
        

Searching keywords and location is successfull :)


In [50]:
time.sleep(3)
LinkedScrapeObject.SearchingForJobs()
   

I am on Page Number 1.....
On Job title : Tech Writing- Virtual Intern or CO-OP 
On Job title : ACCA Trainee 
On Job title : Summer Internship 2022 
On Job title : Ecommerce Developer (Freelancer) 
On Job title : Junior Global Data In Specialist 
On Job title : Business Analyst 
On Job title : Flutter Developer 
On Job title : Next.js and Strapi Developer required. 
On Job title : Computer Science Teacher 
On Job title : Information Technology Assistant 
On Job title : React JS Developer 
On Job title : PHP Developer 
I am on Page Number 2.....
On Job title : Tech Writing- Virtual Intern or CO-OP 
On Job title : ACCA Trainee 
On Job title : Summer Internship 2022 
On Job title : Ecommerce Developer (Freelancer) 
On Job title : Junior Global Data In Specialist 
On Job title : Business Analyst 
On Job title : Flutter Developer 
On Job title : Next.js and Strapi Developer required. 
On Job title : Software Engineer 
On Job title : Computer Science Teacher 
On Job title : Information Techn